In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# 使用 CIFAR-10 数据集作为示例
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
# 选择两个类别用于二分类（如类别 0 和类别 1）
selected_classes = [0, 1]
mask_train = np.isin(y_train, selected_classes).flatten()
mask_test = np.isin(y_test, selected_classes).flatten()

In [4]:
x_train, y_train = x_train[mask_train], y_train[mask_train]
x_test, y_test = x_test[mask_test], y_test[mask_test]

In [5]:
# 正则化像素值
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [6]:
# 转换标签为二分类
y_train = (y_train == selected_classes[1]).astype("int").flatten()
y_test = (y_test == selected_classes[1]).astype("int").flatten()

In [7]:
# 分离有标签数据和无标签数据
x_labeled, x_unlabeled, y_labeled, _ = train_test_split(x_train, y_train, test_size=0.8, random_state=42)

In [18]:
# 修正目标值形状
y_labeled = np.expand_dims(y_labeled, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

In [19]:
# 定义模型
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [20]:
# 构建模型
def build_student_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Conv2D(64, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)  # 输出形状为 (batch_size, 1)
    return Model(inputs, outputs)

In [21]:
# 检查输出
student_model = build_student_model(input_shape=(32, 32, 3))
print(student_model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 16, 16, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         524,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 544,321 (2.08 MB)

 Trainable params: 544,129 (2.08 MB)

 Non-trainable params: 192 (768.00 B)

None


In [22]:
#student_model = build_student_model(input_shape=(32, 32, 3))
teacher_model = tf.keras.models.clone_model(student_model)  # 教师模型初始化为学生模型

In [23]:
# 定义损失函数
def consistency_loss(student_output, teacher_output):
    """
    一致性损失，鼓励学生模型和教师模型的输出相似。
    """
    return tf.reduce_mean(tf.square(student_output - teacher_output))

def supervised_loss(y_true, y_pred):
    """
    有标签数据的监督损失（交叉熵）。
    """
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [24]:
# Mean Teacher 训练流程
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
alpha = 0.99  # Teacher EMA 更新系数

@tf.function
def train_step(x_labeled_batch, y_labeled_batch, x_unlabeled_batch):
    with tf.GradientTape() as tape:
        # 学生模型前向传播
        student_labeled_output = student_model(x_labeled_batch, training=True)
        student_unlabeled_output = student_model(x_unlabeled_batch, training=True)

        # 教师模型前向传播（无梯度更新）
        teacher_unlabeled_output = teacher_model(x_unlabeled_batch, training=False)

        # 损失计算
        sup_loss = supervised_loss(y_labeled_batch, student_labeled_output)
        cons_loss = consistency_loss(student_unlabeled_output, teacher_unlabeled_output)
        total_loss = sup_loss + cons_loss

    # 学生模型反向传播
    gradients = tape.gradient(total_loss, student_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))

    # 教师模型更新 (EMA)
    for t_weights, s_weights in zip(teacher_model.weights, student_model.weights):
        t_weights.assign(alpha * t_weights + (1 - alpha) * s_weights)

    return total_loss

In [25]:
# 模型训练
batch_size = 32
epochs = 10

# 数据分批次
labeled_dataset = tf.data.Dataset.from_tensor_slices((x_labeled, y_labeled)).batch(batch_size)
unlabeled_dataset = tf.data.Dataset.from_tensor_slices(x_unlabeled).batch(batch_size)

In [31]:
print(f"x_labeled_batch shape: {x_labeled_batch.shape}")
print(f"x_unlabeled_batch shape: {x_unlabeled_batch.shape}")

x_labeled_batch shape: (16, 32, 32, 3)
x_unlabeled_batch shape: (32, 32, 32, 3)


In [30]:
# 将有标签和无标签数据集对齐
labeled_dataset = labeled_dataset.batch(batch_size, drop_remainder=True)
unlabeled_dataset = unlabeled_dataset.batch(batch_size, drop_remainder=True)

In [26]:
for epoch in range(epochs):
    total_loss = 0.0
    for (labeled_batch, unlabeled_batch) in zip(labeled_dataset, unlabeled_dataset):
        x_labeled_batch, y_labeled_batch = labeled_batch
        x_unlabeled_batch = unlabeled_batch
        loss = train_step(x_labeled_batch, y_labeled_batch, x_unlabeled_batch)
        total_loss += loss
    print(f"Epoch {epoch + 1}, Loss: {total_loss.numpy():.4f}")

InvalidArgumentError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [32] vs. [16] [Op:AddV2] name: 